In [13]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import feature_extraction as fe

#load everything from images (quite slow) can be skipped, reading features from csv, block 2
kpoints = fe.get_sift_descriptors('roofs/', step=4, ws=8)
print ('Loaded Positives', kpoints.shape)
kpoints = np.concatenate((kpoints,fe.get_sift_descriptors('no_roofs/', step=4, ws=8)))
print ('Loaded Negatives', kpoints.shape)

print (kpoints.shape)

feat_names = ['F'+str(i) for i in range(128)]
data = pd.DataFrame(kpoints,columns=feat_names)
data.to_csv('features.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: 'roofs/'

In [17]:
import cv2
import numpy as np
import pandas as pd
import feature_extraction as fe
from matplotlib import pyplot as plt

data = pd.read_csv('features.csv')
data_arr = np.delete(data.values, 0, axis=1)
'Data loaded from CSV',data_arr.shape

('Data loaded from CSV', (452064, 128))

In [ ]:
from sklearn.cluster import KMeans
from sklearn.externals import joblib
#slow training.use pickled file, next box
k_words = 200
kmeans = KMeans(k_words, max_iter=300, verbose=1, precompute_distances=True, n_init=2, n_jobs=2)
kmeans.fit(data_arr)
joblib.dump(kmeans, '200_words.pkl')
'Model Dumped to File'

NameError: name 'data_arr' is not defined

In [3]:
from sklearn.externals import joblib

kmeans = joblib.load('50_words.pkl') 
kmeans

/Users/francesco/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator KMeans from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=100, n_init=2, n_jobs=2, precompute_distances=True,
    random_state=None, tol=0.0001, verbose=1)

In [44]:
import feature_extraction as fe
import os
from sklearn.preprocessing import normalize

X = []
Y = []
#slow feature extraction problem, load txt files from next box

print ('Starting to load features')
for file in os.listdir('roofs/'):
    if file.endswith('png'):
        img = cv2.imread('roofs/'+file)
        X.append(fe.get_features(img,kmeans))
        Y.append(1)
        
print ('Loaded roofs')      
for file in os.listdir('no_roofs/'):
    if file.endswith('png'):
        img = cv2.imread('no_roofs/'+file)
        X.append(fe.get_features(img,kmeans))
        Y.append(0)
        

X = normalize(np.array(X), norm='max')
Y = np.array(Y).reshape(-1,)

np.savetxt('x50.txt', X, delimiter=',')
np.savetxt('y50.txt', Y, delimiter=',')

print (X.shape, Y.shape)


Starting to load features


Loaded roofs


(961, 100) (961,)


C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [1]:
import numpy as np

X = np.loadtxt('x50.txt', delimiter=',')
Y = np.loadtxt('y50.txt', delimiter=',')

In [5]:
import utils

utils.classifiers_cv_score(X,Y)


[INFO] KNN CV_Score: 0.8010429789353412


[INFO] SVM1 CV_Score: 0.5452657415274237


[INFO] SVM2 CV_Score: 0.6045208774825275


[INFO] RF CV_Score: 0.8372900777025817


[INFO] NN CV_Score: 0.799972004934944
[INFO] GNB CV_Score: 0.7031513979628707
[INFO] Best CV_Score Achieved: 0.8372900777025817 by RF


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=3)
model = MLPClassifier(alpha=1)
model.fit(X_train,Y_train)
Y_pred = np.around(model.predict(X_test))
print (classification_report(Y_test,Y_pred))


             precision    recall  f1-score   support

        0.0       0.81      0.88      0.84        80
        1.0       0.91      0.86      0.88       113

avg / total       0.87      0.87      0.87       193



In [12]:
from utils import simple_ann
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, mode='min')
model = simple_ann(100, 1)
model.compile(loss=['binary_crossentropy'], optimizer='adam',metrics=['accuracy'])
model.fit(X_train,Y_train, epochs=400,batch_size=200,validation_split=0.1, verbose=0)

Y_pred = np.around(model.predict(X_test))
print (classification_report(Y_test,Y_pred))

             precision    recall  f1-score   support

        0.0       0.77      0.88      0.82        80
        1.0       0.90      0.81      0.86       113

avg / total       0.85      0.84      0.84       193



In [11]:
#model = RandomForestClassifier(max_depth=100, n_estimators=1000, max_features='auto')
model = MLPClassifier(alpha=1)
#model =KNeighborsClassifier(3)
model.fit(X,Y)

img = cv2.imread('test2.png')
sq = 40
ind = (len(img) // sq)
sub = []

for i in range(sq):
    for j in range(sq):
        sub.append(img[i*ind:(i + 1) * ind, j*ind:(j + 1) * ind])

roofs = 0
for i in sub:
    if model.predict(fe.get_features(i,kmeans).reshape(1,-1)) == 1:
        roofs += 1
        cv2.imwrite('eval/r'+str(roofs)+'.png', i)
        
roofs

             precision    recall  f1-score   support

        0.0       0.79      0.88      0.83        80
        1.0       0.90      0.83      0.87       113

avg / total       0.86      0.85      0.85       193

